In [0]:
from google.colab import files

# Install Kaggle library
!pip install -q kaggle
uploaded = files.upload()

In [0]:
!chmod 770 /content/kaggle.json

In [0]:
!mkdir /root/.kaggle/
!cp /content/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c ml2019spring-hw3

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

In [0]:
!unzip train.csv.zip
!unzip test.csv.zip

In [0]:
!pip install torch==1.0.1


In [0]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import sys ,os
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


In [0]:
def readfile(path , shuffle):
    print("Reading File...")
    x_train = []
    x_label = []
    val_data = []
    val_label = []

    raw_train = np.genfromtxt(path, delimiter=',', dtype=str, skip_header=1)
    for i in range(len(raw_train)):
        tmp = np.array(raw_train[i, 1].split(' ') ,dtype =float).reshape(1, 48, 48)
        if (i %10 == 0):
            val_data.append(tmp)
            val_label.append(raw_train[i][0])
        else:
          # data augmentation
            
            x_train.append(tmp)
            x_train.append(np.flip(tmp, axis=2))    # simple example of data augmentation  fliplr
            
            noise = np.random.normal(0, 0.03, (1,48,48) )
            mask_overflow_upper = tmp+noise >= 1.0
            mask_overflow_lower = tmp+noise < 0
            noise[mask_overflow_upper] = 1.0
            noise[mask_overflow_lower] = 0
            x_train.append(tmp)
            x_label.append(raw_train[i][0])
            
            x_label.append(raw_train[i][0])
            x_label.append(raw_train[i][0])
    
    x_train = np.array(x_train, dtype=float) / 255.0
    val_data = np.array(val_data, dtype=float) / 255.0
    x_label = np.array(x_label, dtype=int)
    val_label = np.array(val_label, dtype=int)
    x_train = torch.FloatTensor(x_train)
    val_data = torch.FloatTensor(val_data)
    x_label = torch.LongTensor(x_label)
    val_label = torch.LongTensor(val_label)

    return x_train, x_label, val_data, val_label

x_train, x_label, val_data, val_label = readfile('./train.csv' , shuffle=True)    # 'train.csv'

In [0]:
train_set = TensorDataset(x_train, x_label)
val_set = TensorDataset(val_data, val_label)

batch_size = 256
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=32)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=32)

In [0]:
#baseline cnn
def gaussian_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 and classname.find('Conv') == 0:
        m.weight.data.normal_(0.0, 0.02)


class Classifier(nn.Module):


    def __init__(self):
        super(Classifier, self).__init__()
        self.dnn = nn.Sequential(
            nn.Linear(48*48*1 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),


            nn.Linear(1024 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),

            nn.Linear(1024 , 1024) ,
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
        )

        
        
        self.cnn = nn.Sequential(      # output shape
            nn.Conv2d(1, 64, 4 , 2, 1),  # [64, 24, 24]
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, 1, 1),    # [64, 24,24]
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0),      # [64, 12, 12]

            nn.Conv2d(64, 128, 1, 1, 1),    #[64 , 14 , 14]
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 1, 1, 1),    # [64 , 16 , 16]
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0),      # [128, 8, 8]

            nn.Conv2d(128, 256, 1, 1, 1),   # [128, 10, 10]
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 1, 1, 1),    #[128,12,12]
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2, 0),       # [256, 6, 6]
            nn.MaxPool2d(2, 2, 0)       # [256, 3, 3]
            
        )
       
        self.fc = nn.Sequential(
            nn.Linear(256*3*3, 1024), #cnn
            #nn.Linear( 1024,1024), #DNN
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(p=0.5),
            nn.Linear(512, 7),
        )

        self.cnn.apply(gaussian_weights_init)
        #self.dnn.apply(gaussian_weights_init)
        self.fc.apply(gaussian_weights_init)
    def forward(self, x):
        out = self.cnn(x)   # out size = batch size * 256 * 3 * 3 
        out = out.view(out.size()[0], -1) # batch size * (256*3*3)
        
        #x = x.view(x.size()[0],-1)      x.size()[0] = batch size
        #out  = self.dnn(x);
        return self.fc(out)



model = Classifier().cuda()


In [0]:
#Model Construction
def gaussian_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 and classname.find('Conv') == 0:
        m.weight.data.normal_(0.0, 0.02)


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),  # [64, 48, 48]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0),
            nn.Dropout(p=0.3), # [64, 24, 24]

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      
            nn.Dropout(p=0.3),   # [128, 12, 12]
            
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      
            nn.Dropout(p=0.3),   # [256, 6, 6]
            
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      
            nn.Dropout(p=0.4),   # [512, 3, 3]
        )

        self.fc = nn.Sequential(  #fully connected layers
            nn.Linear(512*3*3, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 7),
        )

        self.cnn.apply(gaussian_weights_init)
        self.fc.apply(gaussian_weights_init)

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

#Training
model = Classifier().cuda()
# print(model)


In [0]:
import torch.nn.functional as F

def gaussian_weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 and classname.find('Conv') == 0:
        m.weight.data.normal_(0.0, 0.02)
class ResidualBlock(nn.Module):
    def __init__(self,inchannel , outchannel,stride = 1):
      super(ResidualBlock ,self).__init__()
      '''
      self.left = nn.Sequential(
          nn.Conv2d(inchannel ,outchannel,kernel_size = 3,stride =stride,padding=1),
          nn.BatchNorm2d(outchannel),
          nn.ReLU(),
          nn.Conv2d(outchannel ,outchannel,kernel_size = 3,stride =1,padding=1),
          nn.BatchNorm2d(outchannel)
      )
      '''
      self.left = nn.Sequential(
          nn.Conv2d(inchannel ,outchannel,kernel_size = 1,stride =1,padding=0),
          nn.BatchNorm2d(outchannel),
          nn.ReLU(),
          nn.Conv2d(outchannel ,outchannel,kernel_size = 3,stride =stride,padding=1),
          nn.BatchNorm2d(outchannel),
          nn.ReLU(),
          nn.Conv2d(outchannel ,outchannel,kernel_size = 1,stride =1,padding=0),
          nn.BatchNorm2d(outchannel),
      )
      
      self.shortcut = nn.Sequential()
      if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock,num_classes = 7):
      super(ResNet , self).__init__()
      self.inchannel = 64
      self.conv1 = nn.Sequential(
          nn.Conv2d(1 ,64,kernel_size=3,stride=1,padding =1),  # 64 * 48, 48 
          nn.BatchNorm2d(64),
          nn.ReLU(),
      )
      self.layer1 = self._layer_(ResidualBlock , 64,2,stride = 1)  # [64* 32* 32]
      self.layer2 = self._layer_(ResidualBlock , 128,2,stride = 2)  #[ 64, /2,/2]
      self.layer3 = self._layer_(ResidualBlock , 256,2,stride = 2)
      self.layer4 = self._layer_(ResidualBlock , 512,2,stride = 2)
      self.fc = nn.Linear(512,num_classes)
    def _layer_(self, block , channels , num_blocks  , stride):
      strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
      layers = []
      for stride in strides:
          layers.append(block(self.inchannel, channels, stride))
          self.inchannel = channels
      return nn.Sequential(*layers)
    def forward(self, x):
      out = self.conv1(x)
      out = self.layer1(out)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = F.avg_pool2d(out, 4)
      out = out.view(out.size(0), -1)
      out = self.fc(out)
      return out
def ResNet18():

    return ResNet(ResidualBlock)

model = ResNet18().cuda()
#print(model)   

In [0]:
# print(model)
#model = ResNet18().cuda()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
best_acc = 0.0
num_epoch = 50
val_acc_ = []
train_acc_ = []
model_path = '/content/drive/My'' ''Drive/Colab_model/'
if os.path.isfile(model_path+'model_cnn'):
    model.load_state_dict(torch.load(model_path+'model_cnn'))

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train()
    for i, data in enumerate(train_loader):
        optimizer.zero_grad()
        train_pred = model(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        
        train_loss += batch_loss.item()
        progress = ('#' * int(float(i)/len(train_loader)*40)).ljust(40)
        print ('[%03d/%03d] %2.2f sec(s) | %s |' % (epoch+1, num_epoch, \
                (time.time() - epoch_start_time), progress), end='\r', flush=True)
    
    model.eval()
    for i, data in enumerate(val_loader):
        val_pred = model(data[0].cuda())
        batch_loss = loss(val_pred, data[1].cuda())

        val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        
        val_loss += batch_loss.item()

        progress = ('#' * int(float(i)/len(val_loader)*40)).ljust(40)
        print ('[%03d/%03d] %2.2f sec(s) | %s |' % (epoch+1, num_epoch, \
                (time.time() - epoch_start_time), progress), end='\r', flush=True)

    val_acc = val_acc/val_set.__len__()
    train_acc = train_acc/train_set.__len__()
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc, train_loss, val_acc, val_loss))

    train_acc_.append(train_acc)
    val_acc_.append(val_acc)

    if (val_acc > best_acc):
        with open('./acc.txt','w') as f:
            f.write(str(epoch)+'\t'+str(val_acc)+'\n')
        torch.save(model.state_dict(), model_path+'model_cnn')
        best_acc = val_acc
        print ('Model Saved!')

plt.plot(np.arange(num_epoch),np.expand_dims(train_acc_ ,axis= 1) )
plt.plot(np.arange(num_epoch),np.expand_dims(val_acc_ , axis = 1) )
plt.show()

In [0]:
x_result=[]
x_test=[]
raw_test = np.genfromtxt('./test.csv' , delimiter=',', dtype=str, skip_header=1)
for i in range(len(raw_test)):
  tmp = np.array(raw_test[i,1].split(' ')).reshape(1,48,48)
  tmp =  np.array(tmp , dtype=float)/255.0  
  x_test.append(tmp)

x_test = torch.FloatTensor(  x_test )


In [0]:
x_result=[]
bz = 32
for i in range(1+x_test.shape[0]//bz):
  if i==x_test.shape[0]//bz:
    x_pred = model(x_test[i*bz:].cuda())
    x_result.extend(np.argmax(x_pred.cpu().data.numpy(), axis=1) )
    break
  x_pred = model(x_test[i*bz:(i+1)*bz].cuda())
  x_result.extend(np.argmax(x_pred.cpu().data.numpy(), axis=1) )


In [0]:
import os
if os.path.isfile('./output.csv'):
    os.remove('./output.csv')
with open('./output.csv',"w") as f:
  w = csv.writer(f)
  title = ['id','label']
  w.writerow(title) 
  for i in range(len(x_result)):
      content = [str(i),str(x_result[i]) ]
      
      w.writerow(content) 
    

In [0]:
import sys
import csv
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline  


In [0]:
def readfile(path , shuffle):
    print("Reading File...")
    x_train = []
    x_label = []
    val_data = []
    val_label = []

    raw_train = np.genfromtxt(path, delimiter=',', dtype=str, skip_header=1)
    for i in range(len(raw_train)):
        tmp = np.array(raw_train[i, 1].split(' ') ,dtype =float).reshape(1, 48, 48)
        if (i %10 == 0):
            val_data.append(tmp)
            val_label.append(raw_train[i][0])
        else:
          # data augmentation
            
            x_train.append(tmp)
            x_train.append(np.flip(tmp, axis=2))    # simple example of data augmentation  fliplr
            
            noise = np.random.normal(0, 0.03, (1,48,48) )
            mask_overflow_upper = tmp+noise >= 1.0
            mask_overflow_lower = tmp+noise < 0
            noise[mask_overflow_upper] = 1.0
            noise[mask_overflow_lower] = 0
            x_train.append(tmp)
            x_label.append(raw_train[i][0])
            
            x_label.append(raw_train[i][0])
            x_label.append(raw_train[i][0])
    
    x_train = np.array(x_train, dtype=float) / 255.0
    val_data = np.array(val_data, dtype=float) / 255.0
    x_label = np.array(x_label, dtype=int)
    val_label = np.array(val_label, dtype=int)
    x_train = torch.FloatTensor(x_train)
    val_data = torch.FloatTensor(val_data)
    x_label = torch.LongTensor(x_label)
    val_label = torch.LongTensor(val_label)

    return x_train, x_label, val_data, val_label

x_train, x_label, val_data, val_label = readfile('./train.csv' , shuffle=True)    # 'train.csv'

In [0]:
model = Classifier()
model_path = '/content/drive/My'' ''Drive/Colab_model/model_cnn'
model.load_state_dict(torch.load(model_path))
model.cuda()

In [0]:
def compute_saliency_maps(x, y, model):
    model.eval()
    x.requires_grad_()
    y_pred = model(x.cuda())
    loss_func = torch.nn.CrossEntropyLoss()
    loss = loss_func(y_pred, y.cuda())
    loss.backward()

    saliency = x.grad.abs().squeeze().data
    return saliency

In [0]:
saliency.shape

In [0]:
x_img = x_train[:1].squeeze().numpy()
saliency = compute_saliency_maps(x_train[:1], x_label[:1], model)
saliency = saliency.detach().cpu().numpy()
plt.imshow(x_img, cmap=plt.cm.gray)
plt.show()
plt.imshow(saliency, cmap=plt.cm.jet)
plt.show()

In [0]:
def show_saliency_maps(x, y, model):
    x_org = x.squeeze().numpy()
    # Compute saliency maps for images in X
    saliency = compute_saliency_maps(x, y, model)

    # Convert the saliency map from Torch Tensor to numpy array and show images
    # and saliency maps together.
    saliency = saliency.detach().cpu().numpy()
    num_pics = x_org.shape[0]
    print(num_pics)
    
    for i in range(num_pics):
        # You need to save as the correct fig names
        plt.imshow(x_org[i], cmap=plt.cm.gray)
        plt.show()
        plt.imshow(saliency[i], cmap=plt.cm.jet)
        plt.show()
        #plt.imsave('p3/pic_'+ str(i), x_org[i], cmap=plt.cm.gray)
        #plt.imsave('p3/saliency_'+ str(i), saliency[i], cmap=plt.cm.jet)
    
# using the first ten images for example
show_saliency_maps(x_train[:5], x_label[:5], model)

In [0]:
!pip install lime


In [0]:
import numpy as np
import torch
from lime import lime_image
from skimage.segmentation import slic
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import skimage
from skimage.segmentation import mark_boundaries


In [0]:
for i in range(7):
    for j in range(700):
        if(x_label[j]==i):
            plt.imshow(x_train[j][0], cmap=plt.cm.gray)
            plt.show()
            print(x_label[j])
            print(i , j)
            break

In [0]:



# load data and model
#x_train = torch.load('data/train_data.pth')
#x_label = torch.load('data/train_label.pth')

# Lime needs RGB images
# TODO:

#x_train_rgb = torch.FloatTensor(  x_train_rgb )


# x_train_rgb = ?

model = Classifier()
model_path = '/content/drive/My'' ''Drive/Colab_model/model_cnn'
model.load_state_dict(torch.load(model_path))

# two functions that lime image explainer requires
def predict(input):
    # Input: image tensor
    # Returns a predict function which returns the probabilities of labels ((7,) numpy array)
    # ex: return model(data).numpy()
    # TODO:
    model.eval().cuda()
    input = input.swapaxes(-1,-2)
    input = torch.FloatTensor(  input.swapaxes(-3,-2) )
    #print(input.shape , input.dtype)
    logits = model(input[:,:1,:,:].cuda())
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()
    
    # return ?

def segmentation(input):
    # Input: image numpy array
    # Returns a segmentation function which returns the segmentation labels array ((48,48) numpy array)
    # ex: return skimage.segmentation.slic()
    # TODO:
    # return ?

    return skimage.segmentation.slic(input)

# Initiate explainer instance
explainer = lime_image.LimeImageExplainer()

# Get the explaination of an image

for idx in range(10):
    #x_train_rgb = np.zeros([3,48,48])
    #x_train_rgb[0,:,:] = x_train[idx][0].data
    #x_train_rgb[1,:,:] = x_train[idx][0].data
    #x_train_rgb[2,:,:] = x_train[idx][0].data

    explaination = explainer.explain_instance(
                                image=np.array(x_train[idx].squeeze()  , dtype = float), 
                                classifier_fn=predict,
                                segmentation_fn=segmentation
                            )

    # Get processed image
    image, mask = explaination.get_image_and_mask(
                                    label=int(x_label[idx]),
                                    positive_only=False,
                                    hide_rest=False,
                                    num_features=5,
                                    min_weight=0.0
                                )
    img_boundry1 = mark_boundaries(image, mask)
    plt.imshow(image)
    plt.show()
    plt.imshow(img_boundry1)
    plt.show()
# save the image
#plt.imsave('img', image)